In [ ]:
"""
x_label1 = data[data['Species'] == 'Iris-setosa'].iloc[:, [1, 2, 3, 4]].values
x_label2 = data[data['Species'] == 'Iris-versicolor'].iloc[:, [1, 2, 3, 4]].values
x_label3 = data[data['Species'] == 'Iris-virginica'].iloc[:, [1, 2, 3, 4]].values
x_labels = [x_label1, x_label2, x_label3]
neigh_label1 = NearestNeighbors(n_neighbors=4)
neigh_label2 = NearestNeighbors(n_neighbors=4)
neigh_label3 = NearestNeighbors(n_neighbors=4)
neigh_label1.fit(x_label1)
neigh_label2.fit(x_label2)
neigh_label3.fit(x_label3)
neighs = [neigh_label1, neigh_label2, neigh_label3]"""
"""
def find_neighbors(x, y, neighs, i):
    label_index = labels.tolist().index(y[i])
    neighbors = neighs[label_index].kneighbors(x[i].reshape(1, -1), 4, return_distance=False)[:,1:]
    neighbors = neighbors + 50*label_index
    return neighbors

def delta_function(x, y, neighs, i, j):
    neighbors = find_neighbors(x, y, neighs, i)
    return 1 if j in neighbors else 0
    
def Tv(x, y, neighs, i):
    sum_ = 0
    for j, item in enumerate(x):
        sum_ += (LA.norm(x[i] - x[j])**2)*delta_function(x, y, neighs, i, j)
    return sum_
"""

"""        print("la", LA.norm(x[i] - x[max_index])**2)
        print("m", self.m(i))
        print("t1", t1)
        print(y[i], y[max_index])
        print("t2", t2)
        input("asd")"""

In [208]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.neighbors import NearestNeighbors
from numpy import linalg as LA
from tqdm import tqdm

In [209]:
data = pd.read_csv("IRIS.csv")

In [210]:
x = data.iloc[:, [1, 2, 3, 4]].values
y = data.iloc[:, [5]].values
labels = np.unique(y)

In [239]:
class RBML:
    def __init__(self, x, y, k=3, alpha=0.5, beta=2.):
        self.x = x
        self.y = y
        self.k = k
        self.alpha = alpha
        self.beta = beta
        self.labels = labels = np.unique(self.y)
        self.neigh = NearestNeighbors(n_neighbors=4)
        self.neigh.fit(self.x)
        
    def find_neighbors(self, i):
        neighbors = self.neigh.kneighbors(self.x[i].reshape(1, -1), 4, return_distance=False)[:,1:]
        return neighbors
    
    def delta_t(self, i, j):
        neighbors = self.find_neighbors(i)
        return 1 if j in neighbors else 0
    
    def Tv(self, i):
        sum_ = 0
        for j in range(len(self.x)):
            sum_ += (LA.norm(x[i] - x[j])**2)*self.delta_t(i, j)
        return sum_
    
    def TX(self):
        sum_ = 0
        print("TX calculation started")
        for i in tqdm(range(len(self.x))):
            sum_ += self.Tv(i)
        return sum_
    
    def m(self, i):
        neighbors = self.find_neighbors(i)
        max_ = self.x[neighbors[:,-1]]
        mi = self.beta*(LA.norm(x[i] - max_)**2)*self.delta_t(i, neighbors[:,-1])
        return mi
    
    def delta_i(self, i):
        neighbors = self.find_neighbors(i)
        max_index = neighbors[:,-1]
        t1 = (LA.norm(x[i] - x[max_index])**2 < self.m(i))
        t2 = (y[i] != y[max_index])
        return 1 if (LA.norm(x[i] - x[max_index])**2 < self.m(i)) and (y[i] != y[max_index]) else 0
    
    def Hv(self, i):
        sum_ = 0
        for j in range(len(self.x)):
            sum_ += (self.m(i) - LA.norm(x[i] - x[j])**2)*self.delta_i(i)
        return sum_
    
    def HX(self):
        sum_ = 0
        print("HX calculation started")
        for i in tqdm(range(len(self.x))):
            sum_ += self.Hv(i)
        return sum_
    
    def C(self):
        return (1-self.alpha)*self.TX() + self.alpha*self.HX()
    
    def xiN(self, i):
        sum_ = 0
        sum_t = 0
        for j in range(len(self.x)):
            sum_ += self.x[j]*self.delta_t(i,j)
            sum_t += self.delta_t(i,j)
        
        return sum_/sum_t
    
    def xjI(self, i):
        sum_ = 0
        sum_t = 0
        for j in range(len(self.x)):
            sum_ += self.x[j]*self.delta_i(i)
            sum_t += self.delta_i(i)
        # TODO: DEGISECEK
        return sum_/sum_t if sum_t != 0 else 0
    
    # Updated hinge loss
    def Hv_(self, i):
        return max(0, (self.m(i) - LA.norm(x[i] - self.xjI(i))**2))
    
    def xiH(self, i):
        return self.x[i] + self.m(i)*((self.x[i]-self.xjI(i))/(LA.norm(x[i] - self.xjI(i))**2))  
    
    def xi_star(self, i):
        return (1-self.alpha)*self.xiN(i) + self.alpha*self.xiH(i)

In [237]:
rbml_ = RBML(x, y)

In [238]:
rbml_.xi_star(0)

[15.2 10.5  4.3  0.7] 3
[0. 0. 0. 0.] 0
[0. 0. 0. 0.] 0


array([5.08586687, 3.5017387 , 1.41736215, 0.21676602])

In [240]:
X[0]

array([1. , 5.1, 3.5, 1.4, 0.2])